# Reed-Solomon Berlekamp bit-serial encoder

This is a Sage notebook that calculates the quantities needed in the bit-serial encoder (Berlekamp architecture) encoder for the CCSDS Reed-Solomon code. We use the notation of the report [Perlman, M., Lee J. J., The Reed-Solomon encoders: Conventional versus Berlekamp's architecture](https://ntrs.nasa.gov/citations/19830008870). The motivation of this notebook is to correct an error in the formulas for $T_3$ and $T_4$ in this report.

CCSDS Reed-Solomon code defining parameters:

In [1]:
k.<alpha> = GF(256, modulus=x^8+x^7+x^2+x+1)
K.<x> = k[]
beta = alpha^117
gamma = alpha^11
g = product([x-gamma^j for j in range(112, 143+1)])  # code generator polynomial

Coefficients of $g(x)$ as powers of $\alpha$.
$$g(x) = G_0 + G_1 x + G_2 x^2 + \cdots + G_{32} x^{32}.$$

In [2]:
G_powers = [G.log(alpha) for G in list(g)]
pretty_print(LatexExpr('\\begin{split}'
                       + '\\\\'.join([f'G_{{{j}}} &= G_{{{32-j}}} = \\alpha^{{{n}}}'
                                      for j, n in enumerate(G_powers[:17])])
                       + '\\end{split}'))

\begin{split}G_{0} &= G_{32} = \alpha^{0}\\G_{1} &= G_{31} = \alpha^{249}\\G_{2} &= G_{30} = \alpha^{59}\\G_{3} &= G_{29} = \alpha^{66}\\G_{4} &= G_{28} = \alpha^{4}\\G_{5} &= G_{27} = \alpha^{43}\\G_{6} &= G_{26} = \alpha^{126}\\G_{7} &= G_{25} = \alpha^{251}\\G_{8} &= G_{24} = \alpha^{97}\\G_{9} &= G_{23} = \alpha^{30}\\G_{10} &= G_{22} = \alpha^{3}\\G_{11} &= G_{21} = \alpha^{213}\\G_{12} &= G_{20} = \alpha^{50}\\G_{13} &= G_{19} = \alpha^{66}\\G_{14} &= G_{18} = \alpha^{170}\\G_{15} &= G_{17} = \alpha^{5}\\G_{16} &= G_{16} = \alpha^{24}\end{split}

Calculation of the basis $\{\ell_0, \ell_1, \ldots, \ell_7\}$ dual to $\{\beta^0, \beta^1, \ldots, \beta^7\}$. We express them as powers of $\alpha$.

In [3]:
ells = k.dual_basis([beta^j for j in range(8)])
ell_powers = [ell.log(alpha) for ell in ells]
pretty_print(LatexExpr(',\\quad'.join([f'\\ell_{{{j}}} = \\alpha^{{{n}}}'
                                      for j, n in enumerate(ell_powers)])))

\ell_{0} = \alpha^{125},\quad\ell_{1} = \alpha^{88},\quad\ell_{2} = \alpha^{226},\quad\ell_{3} = \alpha^{163},\quad\ell_{4} = \alpha^{46},\quad\ell_{5} = \alpha^{184},\quad\ell_{6} = \alpha^{67},\quad\ell_{7} = \alpha^{242}

Calculation of the expressions $T_\ell$ of the linear functionals $T_\ell(z) = \operatorname{Tr}(zG_\ell)$, where $z = z_0 \ell_0 + z_1 \ell_1 + \cdots + z_7 \ell_7$.

In [4]:
T_data = [[(ell*G).trace() for ell in ells] for G in list(g)]
pretty_print(LatexExpr(
    '\\begin{split}'
    + '\\\\'.join([f'T_{{{j}}} &=' + '+'.join([f'z_{{{k}}}' for k, a in enumerate(T)
                                              if a == 1])
                   for j, T in enumerate(T_data[:17])])
    + '\\end{split}'))

\begin{split}T_{0} &=z_{0}\\T_{1} &=z_{1}+z_{2}+z_{4}+z_{6}\\T_{2} &=z_{2}+z_{3}\\T_{3} &=z_{0}+z_{2}+z_{4}+z_{5}\\T_{4} &=z_{0}+z_{2}+z_{3}+z_{7}\\T_{5} &=z_{0}+z_{1}+z_{2}+z_{6}+z_{7}\\T_{6} &=z_{0}+z_{1}+z_{5}+z_{6}\\T_{7} &=z_{1}+z_{2}+z_{4}\\T_{8} &=z_{0}+z_{1}+z_{3}+z_{6}+z_{7}\\T_{9} &=z_{0}+z_{2}+z_{3}+z_{4}+z_{5}\\T_{10} &=z_{0}+z_{1}+z_{4}+z_{7}\\T_{11} &=z_{4}\\T_{12} &=z_{0}+z_{1}+z_{2}+z_{3}+z_{4}+z_{5}+z_{6}+z_{7}\\T_{13} &=z_{0}+z_{2}+z_{4}+z_{5}\\T_{14} &=z_{0}+z_{1}+z_{2}+z_{4}+z_{5}+z_{6}\\T_{15} &=z_{0}+z_{1}+z_{3}+z_{5}+z_{7}\\T_{16} &=z_{1}+z_{2}+z_{6}\end{split}

Calculation of the expression $z_8$ of the linear functional $z_8 = \operatorname{Tr}(z\beta^8)$, where $z = z_0 \ell_0 + z_1 \ell_1 + \cdots + z_7 \ell_7$.

In [5]:
z8_data = [(ell*beta^8).trace() for ell in ells]
pretty_print(LatexExpr(f'z_8 ='
                       + '+'.join([f'z_{{{k}}}' for k, a in enumerate(z8_data)
                                   if a == 1])))

z_8 =z_{0}+z_{1}+z_{3}+z_{7}

Compute and print a table mimicking Table 4 (page 42).

In [6]:
alphai = [''.join([str(a) for a in (list((alpha^n).polynomial()) + [0]*8)[:8][::-1]]) for n in range(255)]
trace = [(alpha^n).trace() for n in range(255)]
ellj = [''.join([str((beta^j*alpha^n).trace()) for j in range(8)]) for n in range(255)]

header = [['$n$ of $\\alpha^n$', '$i$ of $\\alpha^i$',
           '$\\operatorname{Tr}(\\alpha^n)$', '$j$ of $\\ell_j$'],
          ['', '76543210', '', '01234567']]
body = [[n, x[0], x[1], x[2]] for n, x in enumerate(zip(alphai, trace, ellj))]
table(header + body)

\(n\) of \(\alpha^n\),\(i\) of \(\alpha^i\),\(\operatorname{Tr}(\alpha^n)\),\(j\) of \(\ell_j\)
,76543210,,01234567
\(0\),00000001,\(0\),01111011
\(1\),00000010,\(1\),10101111
\(2\),00000100,\(1\),10011001
\(3\),00001000,\(1\),11111010
\(4\),00010000,\(1\),10000110
\(5\),00100000,\(1\),11101100
\(6\),01000000,\(1\),11101111
\(7\),10000000,\(1\),10001101
\(8\),10000111,\(1\),11000000
